In [1]:
import pandas as pd
import numpy as np

import math

## лаба 1.2 прогонка

In [5]:
def rev(vec):
    tmp = vec.copy()
    tmp = [int(vec[len(vec)-1-i]) for i in range(len(vec))]
    return tmp

def get_file(x):
    y, vec = [], []
    i = 0
    f = open(x)
    for line in f:
        line = line.split()
        y += line

    f.close()
    y = np.array(y)
    sz = int(((1+len(y)*4)**0.5-1)*0.5)
    mat = np.zeros((sz*(sz+1)))

    for i in range(len(y)):
        mat[i] = int(y[i])

    vec = np.array(mat[::-1][:sz])
    mat = np.array(mat[:-sz])
    mat.shape = (sz,sz)
    vect = rev(vec)
    print(mat, vect)
    return mat, vect

In [6]:
def tdma(mat, vec):
    sz = len(mat)
    x = np.zeros(sz)
    p = np.zeros(sz)
    q = np.zeros(sz)

    
    i = 0
    
    p[i] = -int(mat[i][i+1])/int(mat[i][i])
    q[i] = int(vec[i])/int(mat[i][i])
    
    for i in range(1,sz):
        if i == sz-1: 
            p[i]=0
        else:
            p[i] = -int(mat[i][i+1]) / (int(mat[i][i-1]) * p[i-1] + int(mat[i][i]))
        q[i] = (int(vec[i]) - int(mat[i][i-1]) * q[i-1]) / (int(mat[i][i-1]) * p[i-1] + int(mat[i][i]))
    
    x[sz-1] = q[sz-1]
    for i in range(sz-2,-1,-1):
        x[i] = p[i] * x[i+1] + q[i]

    return x    
    
def progon(x):
    mat, vec = get_file(x)
    res = tdma(mat, vec)
    print ('\n',res)

In [7]:
progon('lab1_2.txt')

[[  8.   4.   0.   0.   0.]
 [ -5.  22.   8.   0.   0.]
 [  0.  -5. -11.   4.   0.]
 [  0.   0.  -9. -15.   1.]
 [  0.   0.   0.   1.   7.]] [48, 125, -43, 18, -23]

 [ 2.9165929   6.1668142   0.4891315  -1.69637061 -3.04337563]


## лаба 1.1 LU-разложение

In [8]:
from random import randint


class MatrixException(Exception):
    pass


def getCofactor(A, tmp, p, q, n):  # Function to get cofactor of A[p][q] in tmp[][]
    i = 0
    j = 0
    # Copying into temporary matrix only those element which are not in given row and column
    for row in range(n):
        for col in range(n):
            if row != p and col != q:
                tmp[i][j] = A[row][col]
                j += 1

                if j == n - 1:
                    j = 0
                    i += 1


def determinant(A, n):
    D = 0
    if n == 1:
        return A[0][0]

    tmp = [[0] * n for _ in range(n)]  # Cofactors
    sign = 1  # Plus or minus

    for i in range(n):
        getCofactor(A, tmp, 0, i, n)
        D += sign * A[0][i] * determinant(tmp, n - 1)

        sign = -sign

    return D


def adjoin(A, n):  # Сопряженная
    adj = [[0] * n for _ in range(n)]
    if n == 1:
        adj[0][0] = 1
        return

    tmp = [[0] * n for _ in range(n)]  # Cofactors
    sign = 1  # Plus or minusn
    for i in range(n):
        for j in range(n):
            getCofactor(A, tmp, i, j, n)  # Cofactor A[i][j]

            if (i + j) % 2 == 0:
                sign = 1
            else:
                sign = -1

            adj[j][i] = (sign) * (determinant(tmp, n - 1))  # transpose of the cofactor matrix

    return adj


def inverse(A, B, n):
    det = determinant(A, n)
    if det == 0:
        print("Singular matrix, can't find its inverse")
        return False

    adj = adjoin(A, n)

    for i in range(n):
        for j in range(n):
            B[i][j] = adj[i][j] / det

    return True


def transpose(A, n):
    B = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            B[i][j] = A[j][i]
    return B


def multi(M1, M2):
    sum = 0  # сумма
    tmp = []  # временная матрица
    ans = []  # конечная матрица
    if len(M2) != len(M1[0]):
        raise MatrixException('Matrix can\'t be multiplied')
    else:
        row1 = len(M1)  # количество строк в первой матрице
        col1 = len(M1[0])  # Количество столбцов в 1
        row2 = col1  # и строк во 2ой матрице
        col2 = len(M2[0])  # количество столбцов во 2ой матрице
        for k in range(0, row1):
            for j in range(0, col2):
                for i in range(0, col1):
                    sum = round(sum + M1[k][i] * M2[i][j], 8)
                tmp.append(sum)
                sum = 0
            ans.append(tmp)
            tmp = []
    return ans


def show(A, n):
    for i in range(0, n):
        for j in range(0, n):
            print("\t", A[i][j], " ", end="")
        print("\n")


def LUP_solve(L, U, pi, b, n):
    x = [0 for i in range(n)]
    y = [0 for i in range(n)]

    for i in range(n):
        summ = 0
        for j in range(i):
            summ += L[i][j] * y[j]

        y[i] = b[pi[i]] - summ

    for i in range(n - 1, -1, -1):
        sec_summ = 0
        for j in range(i + 1, n):
            sec_summ += U[i][j] * x[j]

        x[i] = (y[i] - sec_summ) / U[i][i]

    x = [round(x[i], 5) for i in range(len(x))]
    return x


def LUP_decompose(A, n):
    pi = [i for i in range(n)]

    for k in range(n):
        p = 0
        for i in range(k, n):
            if abs(A[i][k]) > p:
                p = abs(A[i][k])
                tmp_k = i
        if p == 0:
            raise MatrixException('Matrix is degenerate')
        pi[k], pi[tmp_k] = pi[tmp_k], pi[k]

        for i in range(n):
            A[k][i], A[tmp_k][i] = A[tmp_k][i], A[k][i]
        for i in range(k + 1, n):
            A[i][k] = A[i][k] / A[k][k]
            for j in range(k + 1, n):
                A[i][j] = A[i][j] - A[i][k] * A[k][j]
    return pi


def get_LU(A):
    n = len(A)
    L = [[0] * n for i in range(0, n)]
    U = [[0] * n for i in range(0, n)]

    for i in range(n):
        L[i][i] = 1
        for j in range(n):
            if j < i:
                L[i][j] = A[i][j]
            else:
                U[i][j] = A[i][j]
    return L, U

In [9]:
def open_matr_vec(x):
    y, vec = [], []
    i = 0
    f = open(x)
    for line in f:
        line = line.split()
        #for i in range(3):
        #arr = np.array(line)
        #arr.append(arr)
        y += line
    
    f.close()
    y = np.array(y)
    sz = int(((1+len(y)*4)**0.5-1)*0.5)
    mat = np.zeros((sz*(sz+1)))

    for i in range(len(y)):
        mat[i] = int(y[i])

    vec = np.array(mat[::-1][:sz])
    mat = np.array(mat[:-sz])
    mat.shape = (sz,sz)
    vect = rev(vec)
    print(mat, vect)
    return mat, vect
   
def method_lu(x):
    mat, vec = open_matr_vec(x)
    n = len(mat)
    print("Start:")
    show(mat, n)
    print("The Adjoint is :\n")

    inv = [[0] * n for _ in range(n)]

    adj = adjoin(mat, n)
    show(adj, n)

    print("The Inverse is :\n")
    if inverse(mat, inv, n):
        show(inv, n)

    pi = LUP_decompose(mat, n)
    print("A after LUP:")
    show(mat, n)

    L, U = get_LU(mat)
    print("L:")
    show(L, n)
    print("U:")
    show(U, n)
    print("A:\n")
    R = multi(L, U)
    show(R, n)
    print(LUP_solve(L, U, pi, vec, n))

In [12]:
method_lu('lab1_0.txt')

[[ 3. -8.  1. -7.]
 [ 6.  4.  8.  5.]
 [-1.  1. -9. -3.]
 [-6.  6.  9. -4.]] [96, -13, -54, 82]
Start:
	 3.0  	 -8.0  	 1.0  	 -7.0  

	 6.0  	 4.0  	 8.0  	 5.0  

	 -1.0  	 1.0  	 -9.0  	 -3.0  

	 -6.0  	 6.0  	 9.0  	 -4.0  

The Adjoint is :

	 455.0  	 959.0  	 746.0  	 -157.0  

	 -175.0  	 644.0  	 951.0  	 398.0  

	 140.0  	 70.0  	 -510.0  	 225.0  

	 -630.0  	 -315.0  	 -840.0  	 -490.0  

The Inverse is :

	 0.06220095693779904  	 0.13110047846889952  	 0.10198222829801777  	 -0.021462747778537253  

	 -0.023923444976076555  	 0.08803827751196172  	 0.13000683526999315  	 0.054408749145591254  

	 0.019138755980861243  	 0.009569377990430622  	 -0.06971975393028025  	 0.030758714969241284  

	 -0.0861244019138756  	 -0.0430622009569378  	 -0.11483253588516747  	 -0.06698564593301436  

A after LUP:
	 6.0  	 4.0  	 8.0  	 5.0  

	 0.5  	 -10.0  	 -3.0  	 -9.5  

	 -1.0  	 -1.0  	 14.0  	 -8.5  

	 -0.16666666666666666  	 -0.16666666666666666  	 -0.5833333333333334  	 -8.70

## лаба 1.3

### метод простых итераций (Якоби)

In [8]:
def open_matr(x):
    y, vec = [], []
    i = 0
    f = open(x)
    for line in f:
        line = line.split()
        #for i in range(3):
        #arr = np.array(line)
        #arr.append(arr)
        y += line

    f.close()
    y = np.array(y)
    sz = float(((1+len(y)*4)**0.5-1)*0.5)
    mat = np.zeros((sz*(sz+1)))

    for i in range(len(y)):
        mat[i] = int(y[i])

    vec = np.array(mat[::-1][:sz])
    mat = np.array(mat[:-sz])
    mat.shape = (sz,sz)
    print(mat, vec)
    return mat, vec

In [9]:
def open_matr_vec_eps(x):
    y, vec = [], []
    i = 0
    f = open(x)
    for line in f:
        line = line.split()
        #for i in range(3):
        #arr = np.array(line)
        #arr.append(arr)
        y += line

    f.close()
    y = np.array(y)
    epsilon = float(y[-1])
    size = int(((len(y)-1)**0.5))
    #size = int((len(y)-1)/5)
    mat = np.zeros(len(y)-1)
    for i in range(len(y)-1):
        mat[i] = int(y[i])
    vec = np.array(mat[::-1][:size])
    mat = np.array(mat[:-size])
    mat.shape = (size,size)
    print(mat, vec, epsilon)
    return mat, vec, epsilon

In [10]:
mat, vec, epsilon = open_matr_vec_eps('lab1_3.txt')

[[ 20.   5.   7.   1.]
 [ -1.  13.   0.  -7.]
 [  4.  -6.  17.   5.]
 [ -9.   8.   4. -25.]] [ -21.   49.   -1. -117.] 0.01


In [11]:

# поиск значений бета
def find_beta(mat, vec):
    beta1 = []
    beta = np.zeros(len(vec))
    for i in range(len(vec)):
        beta[i] = vec[len(vec)-i-1]/mat[i][i]
    #print(beta)
    return beta

In [12]:
beta = find_beta(mat, vec)
beta

array([-5.85      , -0.07692308,  2.88235294,  0.84      ])

In [13]:
# поиск матрицы альфа
def find_alpha(mat):
    alpha = np.zeros((len(mat),len(mat)))
    for i in range(len(mat)):
        for j in range(len(mat)):
            if i==j:
                alpha[i][j] = 0
            else:    
                alpha[i][j] = -mat[i][j]/mat[i][i]
    return alpha

In [14]:
alpha = find_alpha(mat)
alpha

array([[ 0.        , -0.25      , -0.35      , -0.05      ],
       [ 0.07692308,  0.        , -0.        ,  0.53846154],
       [-0.23529412,  0.35294118,  0.        , -0.29411765],
       [-0.36      ,  0.32      ,  0.16      ,  0.        ]])

In [15]:
def norma_mat(mat):
    norm = np.zeros(len(beta))
    for i in range(len(beta)):
        for j in range(len(beta)):
            norm[i] += mat[i][j]
    norma = max(norm)
    return norma

def norma(x,y):
    summa = 0
    for i in range(len(x)):
        summa += (x[i]-y[i])**2
    res = summa**0.5
    return res

norma_mat(alpha)

0.6153846153846154

In [16]:
def iterasion(x_0,x_1):
    #print(x_0, x_1)
    if (norma_mat(alpha) < 1):
        eps = norma_mat(alpha)/(1-norma_mat(alpha))*norma(x_1, x_0)
    else:
        eps = norma(x_1,x_0)
    if (eps<=epsilon):
        print('final x =',x_1,', epsilon =',eps)
        #break
    else:
        x_0 = x_1
        x_1 = beta + alpha.dot(x_0)
        print('eps =',eps,'>',epsilon)
        iterasion(x_0,x_1)

In [17]:
x_0 = np.copy(beta)
x_1 = beta + alpha.dot(x_0)
iterasion(x_0,x_1)

eps = 4.7311835829058415 > 0.01
eps = 2.5207812740450244 > 0.01
eps = 1.169917752347546 > 0.01
eps = 0.6601266149931471 > 0.01
eps = 0.3542559093189697 > 0.01
eps = 0.20108677554126908 > 0.01
eps = 0.10985606183755829 > 0.01
eps = 0.062431988372992124 > 0.01
eps = 0.034452179840560974 > 0.01
eps = 0.01954162357180484 > 0.01
eps = 0.010865978340489404 > 0.01
final x = [-7.9978861   1.99721773  3.99896611  4.99671502] , epsilon = 0.006150391622279893


### метод Зейделя

In [18]:
#получение матриц В-нижняя треугольная и С-верхняя треугольная

B = np.copy(alpha)
C = np.copy(alpha)

for i in range(len(alpha)):
    for j in range(len(alpha)):
        if (i>j):
            C[i][j] = 0
        else:
            B[i][j] = 0
B, C

(array([[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.07692308,  0.        ,  0.        ,  0.        ],
        [-0.23529412,  0.35294118,  0.        ,  0.        ],
        [-0.36      ,  0.32      ,  0.16      ,  0.        ]]),
 array([[ 0.        , -0.25      , -0.35      , -0.05      ],
        [ 0.        ,  0.        , -0.        ,  0.53846154],
        [ 0.        ,  0.        ,  0.        , -0.29411765],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]))

In [19]:
def new_x(x_0):
    x_0 = np.copy(x_0)
    x = np.zeros(len(x_0))
    for i in range(len(x_0)):
        x[i] = beta[i]
        for j in range(len(x_0)):
            if (i>j):
                x[i] += B[i][j]*x[j]
            else:
                x[i] += C[i][j]*x_0[j]
    return x

In [20]:
def it_Zeydelya(x_0,x):
    print(x_0, x)
    if (norma_mat(alpha) < 1):
        eps = norma_mat(C)/(1-norma_mat(alpha))*norma(x, x_0)
    else:
        eps = norma(x,x_0)
    
    if (eps<=epsilon):
        print('\n FINAL x =',x, 'epsilon =',eps)
    else:
        x_0 = x
        x = new_x(x_0)
        print('eps =',eps,'>',epsilon)
        it_Zeydelya(x_0,x)

In [21]:
x_0 = np.copy(beta)
x = np.zeros(len(x_0))
eps = 0
it_Zeydelya(x_0,x)

[-5.85       -0.07692308  2.88235294  0.84      ] [0. 0. 0. 0.]
eps = 9.206208353554304 > 0.01
[0. 0. 0. 0.] [-5.85       -0.52692308  4.07285068  3.42904072]
eps = 11.098616963355118 > 0.01
[-5.85       -0.52692308  4.07285068  3.42904072] [-7.315219    1.20677431  4.0209599   4.50300021]
eps = 3.516385044485833 > 0.01
[-7.315219    1.20677431  4.0209599   4.50300021] [-7.78417955  1.7489863   4.00680206  4.84306858]
eps = 1.1110080871384063 > 0.01
[-7.78417955  1.7489863   4.00680206  4.84306858] [-7.93178072  1.9207461   4.00213274  4.95042105]
eps = 0.35093365173364643 > 0.01
[-7.93178072  1.9207461   4.00213274  4.95042105] [-7.97845404  1.97496103  4.00067512  4.984339  ]
eps = 0.1108585009839247 > 0.01
[-7.97845404  1.97496103  4.00067512  4.984339  ] [-7.9931935   1.99209073  4.00021314  4.9950528 ]
eps = 0.035018960263187744 > 0.01
[-7.9931935   1.99209073  4.00021314  4.9950528 ] [-7.99784992  1.99750151  4.00006734  4.99843723]
eps = 0.01106216775980048 > 0.01
[-7.99784992  

## лаба 1.4 метод вращений

In [14]:
def open_mat_eps(x):
    y, vec = [], []
    i = 0
    f = open(x)
    for line in f:
        line = line.split()
        #for i in range(3):
        #arr = np.array(line)
        #arr.append(arr)
        y += line

    f.close()
    y = np.array(y)
    epsilon = float(y[-1])
    #sz = int(((1+len(y)*4)**0.5-1)*0.5)
    sz = int(((len(y)-1)**0.5))
    mat = np.zeros(len(y)-1)

    for i in range(len(y)-1):
        mat[i] = int(y[i])

    #vec = np.array(mat[::-1][:sz])
    #mat = np.array(mat[:-sz])
    mat.shape = (sz,sz)
    print(mat, epsilon)
    return mat, epsilon

In [15]:
mat, epsilon = open_mat_eps('lab1_4.txt')

[[ 0. -7.  7.]
 [-7. -9. -5.]
 [ 7. -5. -1.]] 0.1


In [16]:
# поиск максимального внедиагонального элемента
def find_fi(mat):
    max_el = (mat[0][1])
    i_el = 0
    j_el = 1
    for i in range(len(mat)):
        for j in range(len(mat)):
            if (i != j):
                if (abs(max_el) < abs(mat[i][j])):
                    max_el = (mat[i][j])
                    i_el = i
                    j_el = j
    fi = 0.5*math.atan(2*max_el/(mat[i_el][i_el]-mat[j_el][j_el]))
    return fi,i_el,j_el, max_el

def find_mat_U(fi,i_el,j_el,sz):
    U = np.eye(sz)
    U[i_el][i_el] = math.cos(fi)
    U[j_el][j_el] = math.cos(fi)
    U[i_el][j_el] = -math.sin(fi)
    U[j_el][i_el] = math.sin(fi)   
    #U = np.around(U, decimals=2)
    return U

def find_A(mat, U):
    #A = U.transpose().dot(mat).dot(U)
    t = U.transpose().dot(mat)
    A = t.dot(U)
    #A = np.around(A, decimals=2)
    return A
        

def stop(A,i_el,j_el):
    summ = 0
    if (i_el<j_el): #Значит верхняя треугольная
        for i in range(len(A)):
            for j in range(len(A)):
                if (i<j):
                    summ += A[i][j]**2
        eps = summ**0.5
    if (i_el>j_el): #Значит нижняя треугольная
        for i in range(len(A)):
            for j in range(len(A)):
                if (i>j):
                    summ += A[i][j]**2
        eps = summ**0.5
    return eps  


def method_vrash(mat, epsilon, it, z):
    
    fi,i_el,j_el,q = find_fi(mat)
    U = find_mat_U(fi,i_el,j_el,len(mat))
    U = np.around(U,decimals=2)
    z = z.dot(U)
    A = find_A(mat, U)
    eps = stop(A,i_el,j_el)
    #A = np.around(A, decimals=4)
    if (eps > epsilon):
        #print(it, i_el, j_el, A)
        it += 1
        method_vrash(A, epsilon, it, z)
    else:
        #print('FINAL A on it',it, A, 'eps =', eps)
        for i in range(len(z)):
            print ("x",i+1,"=",z.T[i], "\n")
    
    
def sobstv(a, epsilon):
    z = np.eye(len(a))
    method_vrash(a,epsilon,1,z)

In [17]:
sobstv(mat, epsilon)

x 1 = [ 0.6861728 -0.4099008  0.60432  ] 

x 2 = [0.41712 0.90288 0.138  ] 

x 3 = [-0.6016412   0.15723432  0.786036  ] 



## лаба 1.5 QR

In [18]:
mat, epsilon = open_mat_eps('lab1_5.txt')

[[ 5.  8. -2.]
 [ 7. -2. -4.]
 [ 5.  8. -1.]] 0.3


In [19]:
def find_nu(mat,j):
    nu_temp = 0
    nu = np.copy(mat.T[j])
    for i in range(len(mat)):
        if (j<=i):
            nu_temp += mat[i][j]**2
    nu[j] += (nu[j]/abs(nu[j]))*nu_temp**0.5
    nu = np.around(nu, decimals=2)
    return nu

def mat_Haus(nu):
    H = np.eye(len(nu))
    nu.shape = (3,1)
    H -= 2*(nu.dot(nu.T)/(nu.T.dot(nu)))
    H = np.around(H, decimals=2)
    return H

def mat_A(h,a):
    a = h.dot(a)
    a = np.around(a, decimals=2)
    return a

In [20]:
def find_qr(mat):
    nu1 = find_nu(mat,0)
    H1 = mat_Haus(nu1)
    mat1 = mat_A(H1,mat)
    
    nu2 = find_nu(mat1,1)
    nu2[0] = 0.0 # необходимо обнулить
    H2  = mat_Haus(nu2)
    mat2 = mat_A(H2,mat1)
    
    #Q = np.eye(len(mat))
    Q = np.around(H1.dot(H2),decimals=2)
    
    return mat2, Q

In [21]:
R, Q = find_qr(mat)
np.around(find_qr(mat), decimals=1)

array([[[-9.9, -6.6,  4.3],
        [-0. ,  9.4,  1.4],
        [-0. ,  0. ,  0.7]],

       [[-0.5,  0.5, -0.7],
        [-0.7, -0.7, -0. ],
        [-0.5,  0.5,  0.7]]])

In [22]:
#from matrix import Matrix, Vector
from numpy.linalg import eig
from numpy.linalg import norm

def get_roots(A, i):
    sz = len(A)
    a11 = A[i][i]
    a12 = A[i][i + 1] if i + 1 < sz else 0
    a21 = A[i + 1][i] if i + 1 < sz else 0
    a22 = A[i + 1][i + 1] if i + 1 < sz else 0
    return np.roots((1, -a11 - a22, a11 * a22 - a12 * a21))


def finish_iter_for_complex(A, eps, i):
    r, q = find_qr(A)
    A1 = np.around(r.dot(q),decimals=2)
    lambda1 = np.around(get_roots(A, i),decimals=2)
    lambda2 = np.around(get_roots(A1, i),decimals=2)
    return True if abs(lambda1[0] - lambda2[0]) <= eps and \
                   abs(lambda1[1] - lambda2[1]) <= eps else False


def get_eigenvalue(A, eps, i):
    A_i = np.copy(A)
    #A_i = Matrix(A)
    while True:
        r, q = find_qr(A_i)
        A_i = np.around(r.dot(q),decimals=2)
        #a = np.array(A_i.get_data())
        a = np.copy(A_i)
        if norm(a[i + 1:, i]) <= eps:
            res = (a[i][i], False, A_i)
            break
        elif norm(a[i + 2:, i]) <= eps and finish_iter_for_complex(A_i, eps, i):
            res = (get_roots(A_i, i), True, A_i)
            break
    return res


def method_qr(A, eps):
    res = []
    i = 0
    A_i = np.copy(A)
    while i < len(A):
        eigenval = get_eigenvalue(A_i, eps, i)
        if eigenval[1]:
            res.extend(eigenval[0])
            i += 2
        else:
            res.append(eigenval[0])
            i += 1
        A_i = eigenval[2]
    return res, i

In [23]:
method_qr(mat,0.1)

([-5.380649183969009, 4.940649183969009, 2.58], 3)

In [32]:
def method_qr(mat,i):
    r,q = find_qr(mat)
    A = np.around(r.dot(q),decimals=2)
    print(A)
    if (i==6):
        return
    method_qr(A,i+1)

In [33]:
nu = find_nu(mat,0)
H1 = mat_Haus(nu)
mat1 = mat_A(H1,mat)
nu2 = find_nu(mat1,1)
nu2[0] = 0.0 # необходимо обнулить
H2  = mat_Haus(nu2)
mat2 = mat_A(H2,mat1)
mat2

array([[-4.24, -3.78, -2.14],
       [ 0.  ,  2.18,  0.89],
       [ 0.  , -0.01, -3.58]])

In [34]:
#n.shape = (3,1)
h.dot(mat)

NameError: name 'h' is not defined

In [ ]:
h = np.around(h, decimals=2)
h,mat

In [ ]:
s = h.dot(mat)
s

In [ ]:
z = np.eye(len(s))
z = z.dot(s)
z